In [4]:
import dash
#pip install dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html

#import necessary libraries
from plotly import tools
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go #for calling graph objects from plotly library.
import pandas as pd
import numpy as np 
import requests
#!pip install wget
from datetime import datetime
#from scipy.interpolate import interp1d

     |████████████████████████████████| 107 kB 1.4 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
ERROR: Could not find a version that satisfies the requirement dbc (from versions: none)
ERROR: No matching distribution found for dbc


In [7]:
# loading the dataset
death_df =  pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [8]:
death_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,246,249,257,265,270,294,300,309,327,357
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,33,33,33,33,33,33,33,33,34,34
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,638,646,653,661,667,673,681,690,698,707
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,51,51,51,51,51,51,51,51,51,51
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,4


In [9]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,13659,14525,15205,15750,16509,17267,18054,18969,19551,20342
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1099,1122,1137,1143,1164,1184,1197,1212,1232,1246
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,9134,9267,9394,9513,9626,9733,9831,9935,10050,10154
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,764,764,764,765,844,851,852,852,852,852
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,81,84,86,86,86,86,86,86,88,91


In [11]:
recovered_df.head()


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1259,1303,1328,1428,1450,1522,1585,1762,1830,1875
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,851,857,872,877,891,898,898,910,925,938
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,5422,5549,5748,5894,6067,6218,6297,6453,6631,6717
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,684,692,694,698,733,735,738,741,741,744
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,18,18,18,18,18,18,18,21,24,24


In [7]:
country_df.head()

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Australia,2020-06-06 18:33:04,-25.0000,133.0000,7255.0,102.0,6693.0,460.0,28.496015,NaN,NaN,1.405927,36,AUS
1,Austria,2020-06-06 18:33:04,47.5162,14.5501,16898.0,672.0,15789.0,437.0,187.622135,NaN,NaN,3.976802,40,AUT
2,Canada,2020-06-06 18:33:04,60.0010,-95.0010,96265.0,7814.0,53844.0,34607.0,254.294584,NaN,NaN,8.117177,124,CAN
3,China,2020-06-06 18:33:04,30.5928,114.3055,84181.0,4638.0,79423.0,120.0,5.992911,NaN,NaN,5.509557,156,CHN
4,Denmark,2020-06-06 18:33:04,56.2639,9.5018,12124.0,587.0,10921.0,616.0,209.315868,NaN,NaN,4.841636,208,DNK


In [13]:
confirmed_df.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '5/29/20', '5/30/20', '5/31/20', '6/1/20', '6/2/20', '6/3/20', '6/4/20',
       '6/5/20', '6/6/20', '6/7/20'],
      dtype='object', length=142)

In [8]:
death_df.isna().sum()

Province/State    185
Country/Region      0
Lat                 0
Long                0
1/22/20             0
                 ... 
6/1/20              0
6/2/20              0
6/3/20              0
6/4/20              0
6/5/20              0
Length: 140, dtype: int64

In [9]:
confirmed_df.isna().sum()

Province/State    185
Country/Region      0
Lat                 0
Long                0
1/22/20             0
                 ... 
6/1/20              0
6/2/20              0
6/3/20              0
6/4/20              0
6/5/20              0
Length: 140, dtype: int64

In [10]:
recovered_df.isna().sum()

Province/State    186
Country/Region      0
Lat                 0
Long                0
1/22/20             0
                 ... 
6/1/20              0
6/2/20              0
6/3/20              0
6/4/20              0
6/5/20              0
Length: 140, dtype: int64

In [11]:
country_df.isna().sum()

Country_Region           0
Last_Update              1
Lat                      2
Long_                    2
Confirmed                1
Deaths                   1
Recovered                3
Active                   1
Incident_Rate            3
People_Tested          188
People_Hospitalized    188
Mortality_Rate           1
UID                      0
ISO3                     2
dtype: int64

In [16]:
#transpose all date columns into values (use melt to unpivot dataframes from wide to long format)
dates = confirmed_df.columns[4:]
confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
death_df_long = death_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

In [17]:
#example of above
confirmed_df_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
36703,NaN,Sao Tome and Principe,0.186360,6.613081,6/7/20,513
36704,NaN,Yemen,15.552727,48.516388,6/7/20,484
36705,NaN,Comoros,-11.645500,43.333300,6/7/20,141
36706,NaN,Tajikistan,38.861034,71.276093,6/7/20,4529


In [18]:
#Canada recovered data is counted by country vs province/state
#remove recovered data for Canada for above mismatch issue
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']

In [20]:
#merge the dataframes
# Merging confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=death_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)
# Merging full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

In [23]:
#convert dataframe from string to datetime 
full_table['Date'] = pd.to_datetime(full_table['Date'])

In [24]:
#see how many NaN values are in merged df 
full_table.isna().sum()

Province/State    25530
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered          3864
dtype: int64